In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

/usr/local/lib/python2.7/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df_x = pd.read_csv("train.csv")
print(df_x.shape)
df_y = pd.DataFrame(columns=["Severity"])
df_y["Severity"] = df_x.loc[:,"Severity"].astype('category')
del df_x["Severity"]
del df_x["Accident_ID"]
trans_df_x = pd.DataFrame(columns=df_x.columns)
trans_df_y = pd.DataFrame(columns=df_y.columns)
trans_df_x = pd.get_dummies(df_x, columns=["Accident_Type_Code"])
trans_df_y = pd.get_dummies(df_y, columns=["Severity"])

(10000, 12)


In [3]:
tr_x = df_x.iloc[:8000,:]
te_x = df_x.iloc[8001:,:]
tr_y = df_y.iloc[:8000,:]
te_y = df_y.iloc[8001:,:]

In [7]:
lr = LogisticRegressionCV(n_jobs=-1).fit(trans_df_x, trans_df_y.values.ravel())

ValueError: Found input variables with inconsistent numbers of samples: [10000, 40000]

In [9]:
neigh1 = KNeighborsClassifier(n_neighbors=5)
neigh1.fit(tr_x, tr_y.values.ravel())
pr_y = neigh1.predict(te_x)
print (f1_score(te_y,pr_y,average='weighted'))

0.4201460192914917


In [10]:
neigh2 = KNeighborsClassifier(n_neighbors=1)
neigh2.fit(tr_x, tr_y.values.ravel())
pr_y = neigh2.predict(te_x)
print (f1_score(te_y,pr_y,average='weighted'))

0.515112139168231


In [11]:
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
mlp.fit(tr_x,tr_y)
pr_y = mlp.predict(te_x)
print (f1_score(te_y,pr_y,average='weighted'))

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.1329082336139056


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
rf = RandomForestClassifier(max_depth=25)
rf.fit(tr_x, tr_y.values.ravel())
pr_y = rf.predict(te_x)
print (f1_score(te_y,pr_y,average='weighted'))

0.9089023033760331


In [13]:
dt = DecisionTreeClassifier()
dt.fit(df_x, df_y.values.ravel())
pr_y = dt.predict(te_x)
print (f1_score(te_y,pr_y,average='weighted'))

1.0


In [49]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(4, input_dim=16, activation="relu"),
  tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

temp = df_y["Severity"].cat.codes
model.fit(trans_df_x, temp, epochs=5)
model.summary()


Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 1s 102us/sample - loss: 1.3762 - accuracy: 0.3037
Epoch 2/5
10000/10000 [==============================] - 1s 81us/sample - loss: 1.3679 - accuracy: 0.3049
Epoch 3/5
10000/10000 [==============================] - 1s 71us/sample - loss: 1.3659 - accuracy: 0.3049
Epoch 4/5
10000/10000 [==============================] - 1s 69us/sample - loss: 1.3653 - accuracy: 0.3049
Epoch 5/5
10000/10000 [==============================] - 1s 71us/sample - loss: 1.3651 - accuracy: 0.3049
Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_106 (Dense)            (None, 4)                 68        
_________________________________________________________________
dense_107 (Dense)            (None, 4)                 20        
Total params: 88
Trainable params: 88
Non-trainable params: 0
_______________________________

In [52]:
fte_x = pd.read_csv("test.csv")
final = pd.DataFrame(columns=["Accident_ID","Severity"])
final["Accident_ID"] = fte_x.loc[:,"Accident_ID"]
del fte_x["Accident_ID"]

final["Severity"] = dt.predict(fte_x)
final.to_csv('solution.csv', header=True, index=False)